In [ ]:
from openai import OpenAI

client = OpenAI(api_key="0", base_url="")

def llm_qwen(prompt, model="Qwen3", temperature=0.7, top_p=0.8, max_tokens=1024, presence_penalty=1.5):
    chat_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
        presence_penalty=presence_penalty,
    )
    return chat_response.choices[0].message.content

if __name__ == "__main__":
    llm_output = llm_qwen("你好")
    print(llm_output)

你好！有什么我可以帮你的吗？😊


In [ ]:
import torch
import concurrent.futures
import re
import json
from typing import List, Tuple, Dict, Optional
from openai import OpenAI

class EnhancedAggregationLayer(torch.nn.Module):
    """
    增强型聚合层，用于多视角响应的语义聚合
    融合ParScale技术的注意力平滑机制，提升多视角分析的稳定性
    """
    def __init__(self, hidden_size=768, num_perspectives=3, parscale_smoothing=0.1):
        """
        初始化聚合层
        
        Args:
            hidden_size: 编码器输出的隐藏层维度，默认768（BERT-base配置）
            num_perspectives: 多视角分析的视角数量，默认3个视角
            parscale_smoothing: ParScale注意力平滑系数，防止极端权重，默认0.1
        """
        super().__init__()
        self.num_perspectives = num_perspectives
        self.hidden_size = hidden_size
        self.parscale_smoothing = parscale_smoothing
        
        # 定义聚合网络结构：线性变换+层归一化+激活函数+Dropout+输出权重
        self.aggregate_layer = torch.nn.Sequential(
            torch.nn.Linear(hidden_size * num_perspectives, hidden_size),
            torch.nn.LayerNorm(hidden_size),
            torch.nn.SiLU(),  # 平滑激活函数，替代ReLU
            torch.nn.Dropout(0.1),  # 防止过拟合
            torch.nn.Linear(hidden_size, num_perspectives)
        )
        self.softmax = torch.nn.Softmax(dim=-1)  # 权重归一化
        
    def forward(self, encoded_responses):
        """
        前向传播：计算多视角响应的聚合权重和综合表示
        
        Args:
            encoded_responses: 各视角响应的编码向量列表，形状为[p, h]
        
        Returns:
            aggregated_output: 聚合后的综合向量，形状为[h]
            weights: 各视角的注意力权重，形状为[1, p]
        """
        stacked_responses = torch.stack(encoded_responses)  # 堆叠视角编码为[p, h]
        concat_responses = stacked_responses.view(1, -1)  # 拼接为[1, p*h]用于特征提取
        
        raw_weights = self.aggregate_layer(concat_responses)  # 计算原始权重
        weights = self.softmax(raw_weights)  # 权重归一化
        
        # 应用ParScale注意力平滑：防止某个视角权重过大
        if self.parscale_smoothing > 0:
            uniform_weight = 1.0 / self.num_perspectives
            weights = weights * (1 - self.parscale_smoothing) + self.parscale_smoothing * uniform_weight
        
        # 加权求和生成综合表示
        weighted_sum = torch.sum(stacked_responses * weights.t(), dim=0)  # [p, h] * [p, 1] 后求和
        return weighted_sum, weights


# 初始化OpenAI客户端，连接到本地Qwen模型服务
client = OpenAI(api_key="0", base_url="http://192.168.106.26:20000/v1")

def llm_qwen(prompt, model="Qwen3", temperature=0.7, top_p=0.8, max_tokens=1024):
    """
    调用Qwen模型API生成响应
    
    Args:
        prompt: 输入提示词
        model: 模型名称，默认Qwen3
        temperature: 生成温度，控制随机性，默认0.7
        top_p: 核采样参数，控制生成多样性，默认0.8
        max_tokens: 最大生成长度，默认1024
    
    Returns:
        模型生成的文本响应
    """
    chat_response = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "user", "content": prompt},
        ],
        temperature=temperature,
        top_p=top_p,
        max_tokens=max_tokens,
    )
    return chat_response.choices[0].message.content


def load_encoder_model():
    """
    加载文本编码器模型（BERT-base）用于语义向量化
    
    Returns:
        model: BERT模型实例
        tokenizer: 对应的分词器
    """
    from transformers import AutoModel, AutoTokenizer
    model_name = "/app/sda1/xiangyue/model/bert-base-chinese"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModel.from_pretrained(model_name)
    return model, tokenizer


def encode_text(text, model, tokenizer):
    """
    将文本转换为语义向量表示（使用BERT的[CLS]标记）
    
    Args:
        text: 输入文本
        model: BERT模型实例
        tokenizer: 分词器实例
    
    Returns:
        文本的语义编码向量，形状为[hidden_size]
    """
    inputs = tokenizer(text, return_tensors="pt", truncation=True, max_length=512)
    with torch.no_grad():  # 推理时关闭梯度计算
        outputs = model(**inputs)
    # 使用BERT的第一个标记（[CLS]）作为整体语义表示
    return outputs.last_hidden_state[:, 0, :].squeeze(0)


def process_perspective(index, question, encoder_model, tokenizer, llm_qwen):
    """
    处理单个视角：生成响应并进行语义编码
    
    Args:
        index: 视角索引（从0开始）
        question: 分析问题
        encoder_model: 编码器模型
        tokenizer: 分词器
        llm_qwen: LLM调用函数
    
    Returns:
        index: 视角索引
        response: 模型生成的响应文本
        encoded_response: 响应的语义编码
    """
    # 为每个视角添加标识，引导模型从不同角度思考
    perspective_prompt = f"【视角{index+1}】{question}"
    response = llm_qwen(perspective_prompt)  # 调用Qwen模型生成响应
    encoded_response = encode_text(response, encoder_model, tokenizer)  # 编码响应文本
    return index, response, encoded_response


def multi_perspective_analysis(metaprompt, p=3, topk=1):
    """
    多视角分析主函数：生成多个视角响应并聚合分析结果
    
    Args:
        metaprompt: 元提示词（分析问题）
        p: 生成的视角数量，默认3个
        topk: 保留的关键视角数量，默认1个
    
    Returns:
        包含聚合结果、关键视角和所有视角权重的字典
    """
    # 初始化聚合模型与编码器
    aggregation_model = EnhancedAggregationLayer(hidden_size=768, num_perspectives=p)
    encoder_model, tokenizer = load_encoder_model()
    
    # 并行处理多个视角（使用线程池提高效率）
    with concurrent.futures.ThreadPoolExecutor(max_workers=p) as executor:
        futures = [
            executor.submit(
                process_perspective, i, metaprompt, encoder_model, tokenizer, llm_qwen
            )
            for i in range(p)
        ]
        # 收集所有视角的处理结果
        results = [future.result() for future in concurrent.futures.as_completed(futures)]
    
    # 按视角索引排序结果
    results.sort(key=lambda x: x[0])
    # 提取响应文本和编码向量
    responses = [(f"response_{i+1}", resp) for i, (_, resp, _) in enumerate(results)]
    encoded_responses = [enc for _, _, enc in results]
    response_texts = [resp for _, resp, _ in results]
    
    # 打印原始响应（调试和可视化）
    print("=== 原始响应 ===")
    for i, (key, response) in enumerate(responses):
        print(f"{key}: {response}")
    
    # 执行聚合逻辑（仅当有有效响应时）
    if encoded_responses:
        aggregated_output, weights = aggregation_model(encoded_responses)
        
        # 打印各视角的聚合权重
        print("\n=== 聚合权重 ===")
        for i, (key, _) in enumerate(responses):
            print(f"{key}: {weights[0, i].item():.4f}")
        
        # 获取权重最高的topk个视角
        top_weights, top_indices = torch.topk(weights[0], topk)
        print(f"\n=== Top {topk} 视角 ===")
        top_perspectives = []
        for i, idx in enumerate(top_indices):
            key = responses[idx][0]
            weight = top_weights[i].item()
            content = response_texts[idx]
            print(f"{key}: 权重={weight:.4f}\n内容: {content}")
            top_perspectives.append((key, weight, content))
        
        # 生成最终聚合结果（合并关键视角内容）
        final_title = "\n".join(re.sub(r'【[^】]+】', '', content).strip() for _, _, content in top_perspectives)
        print("\n=== 最终聚合结果 ===")
        print(final_title)
        
        return {
            "final_title": final_title,
            "top_perspectives": top_perspectives,
            "all_weights": [(responses[i][0], weights[0, i].item()) for i in range(p)]
        }
    else:
        print("没有有效的回答可供聚合")
        return None


if __name__ == "__main__":
    # 示例元提示词（空提示用于测试基本功能）
    metaprompt = """
/no_think
"""
    # 执行多视角分析（生成3个视角，保留1个关键视角）
    result = multi_perspective_analysis(metaprompt, p=3, topk=1)